In [19]:
import configparser
from pyspark.sql import SparkSession
from pyspark.s

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'psycopg2'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'psycopg2'



In [32]:
#Creates spark session and spark context
spark_session = SparkSession.builder.getOrCreate()
sc = spark_session.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Install psycopg for further connection with redshift cluster
sc.install_pypi_package("psycopg2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



# Data quality checks
Data quality checks includes:
- Data schema from s3 bucket
- No empty table in Redshift

## Data schema for fact and dimensional tables

In [34]:
FACT_TABLE_CNPJ = 's3://rf-udct-zen/cnpjs_fact_table_csv/'
CITIES_DATA = 's3://rf-udct-zen/dimensions/cities_lat_long.csv'
CNAES_DATA = 's3://rf-udct-zen/dimensions/cnaes.csv'

tables_path = [FACT_TABLE_CNPJ, CITIES_DATA, CNAES_DATA]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
for path in tables_path:
    print('-'*8)
    table_df = spark.read.option("delimiter",';')\
                        .option("encoding",'iso-8859-1')\
                        .option('header', 'true')\
                        .option('inferSchema', 'true')\
                        .csv(path)
    table_df.printSchema()
    print('-'*8)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------
root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj: long (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- dt_inicioatividade: timestamp (nullable = true)
 |-- cod_cnae: integer (nullable = true)
 |-- cep: integer (nullable = true)
 |-- cod_municipio: integer (nullable = true)
 |-- porte: string (nullable = true)
 |-- phone_contact: string (nullable = true)
 |-- contact_email: string (nullable = true)

--------
--------
root
 |-- codigo_ibge: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- capital: integer (nullable = true)
 |-- codigo_uf: integer (nullable = true)
 |-- siafi_id: integer (nullable = true)
 |-- ddd: integer (nullable = true)
 |-- fuso_horario: string (nullable = true)

--------
--------
root
 |-- cod_cnae: integer (nullable = true)
 |-- cnae_descricao: string (nullable = true)

--------

As can be seen Pyspark has set cnpj_basico and cnpj as integer, what makes sense because these fields contains only numbers. In our scenario, both of them are strings. In this case, is important to set the correct data type in CREATE TABLE SCHEMA. Considering that CNPJ is a formatted number with 14 digits, we cannot remove zeros in the beginning and end of string, because it would make some CNPJs contain less then 14 digits.

In [81]:
fact_table_df = spark.read.option("delimiter",';')\
                        .option("dateFormat",'yyyy-MM-dd')\
                        .option("encoding",'iso-8859-1')\
                        .option('header', 'true')\
                        .option('inferSchema', 'true')\
                        .csv(FACT_TABLE_CNPJ)
dim_table_cnae_df = spark.read.option("delimiter",';')\
                        .option("encoding",'iso-8859-1')\
                        .option('header', 'true')\
                        .csv(CNAES_DATA)
dim_table_cities_df = spark.read.option("delimiter",';')\
                        .option('header', 'true')\
                        .csv(CITIES_DATA)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Checking the total number of rows between S3 files and Redshift

In [78]:
import configparser
import psycopg2 

#AWS CREDENTIALS
HOST='redshift-cluster-1.cjx8xcymz68e.us-east-1.redshift.amazonaws.com'
DB_NAME='dev'
DB_USER='awsuser'
DB_PASSWORD='Dataengineer1452'
DB_PORT='5439'

conn = psycopg2.connect(f"host={HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={DB_PORT}")
cur = conn.cursor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# Total number of records in amazon S3 fact table
number_of_records_s3 = fact_table_df.count()

# Total number of records in amazon Redshift fact table
cur.execute('SELECT COUNT(*) FROM cnpj_receita_federal')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
print(f'total number of records on Redshift (Fact Table): {cur.fetchone()[0]}')
print(f'total number of records on s3 files (Fact Table): {number_of_records_s3}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total number of records on Redshift (Fact Table): 864471
total number of records on s3 files (Fact Table): 864471

#### Checking dimension tables

In [82]:
# Total number of records in amazon S3 dim table
number_of_records_s3 = dim_table_cnae_df.count()

# Total number of records in amazon Redshift fact table
cur.execute('SELECT COUNT(*) FROM cnaes')

print(f'total number of records on Redshift (CNAE table): {cur.fetchone()[0]}')
print(f'total number of records on s3 files (CNAE Table): {number_of_records_s3}')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total number of records on Redshift (CNAE table): 1359
total number of records on s3 files (CNAE Table): 1359

In [84]:
# Total number of records in amazon S3 dim table
number_of_records_s3 = dim_table_cities_df.count()

# Total number of records in amazon Redshift fact table
cur.execute('SELECT COUNT(*) FROM cities')

print(f'total number of records on Redshift (CITIES table): {cur.fetchone()[0]}')
print(f'total number of records on s3 files (CITIES Table): {number_of_records_s3}')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total number of records on Redshift (CITIES table): 5570
total number of records on s3 files (CITIES Table): 5570